## 1. 데이터불러오기

In [199]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import os
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

%matplotlib inline

In [200]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 의안데이터

In [201]:
BillDf_20 = pd.read_csv('/content/drive/My Drive/cuai-conf/20th.csv', parse_dates=['제안일자','의결일자'])
BillDf_21 = pd.read_csv("/content/drive/My Drive/cuai-conf/21대 국회 의안정보 new.csv", parse_dates=['제안일자','의결일자'], encoding='utf-8-sig') #200824업데이트
BillDf_20.head()

,Unnamed: 0,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,index
0,0,ZZ23959,계,「인공지능산업 진흥에 관한 법률안」에 대한 의견표명 결정문,기타,2020-05-28,NaT,NaN,NaN,의안원문,제20대 (2016~2020) 제378회,NaN,NaN,NaN,NaN
1,1,2024996,처,집합건물의 소유 및 관리에 관한 법률 일부개정법률안(김병관의원 등 13인),의원,2020-05-22,2020-05-29,임기만료폐기,NaN,의안원문,제20대 (2016~2020) 제378회,['■ 제안이유아파트 등 공동주택은 「공동주택관리법」에 의해 체계적으로 관리되고 있...,법제사법위원회,"['김병관(더불어민주당/金炳官)', '고용진(더불어민주당/高榕禛)', '김병욱(더불...",NaN
2,2,2024995,처,지방세법 일부개정법률안(김병관의원 등 10인),의원,2020-05-22,2020-05-29,임기만료폐기,NaN,의안원문\n비용추계서 미첨부 사유서,제20대 (2016~2020) 제378회,['제안이유 및 주요내용현행법상 아파트 등 주택에 대한 재산세 과세표준은 국토교통부...,행정안전위원회,"['김병관(더불어민주당/金炳官)', '김철민(더불어민주당/金哲玟)', '박재호(더불...",NaN
3,3,2024994,처,법률용어 정비를 위한 정보위원회 소관 2개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,NaN,의안원문,제20대 (2016~2020) 제378회,NaN,정보위원회,NaN,NaN
4,4,2024993,처,진실·화해를 위한 과거사정리 기본법 일부개정법률안(대안),위원장,2020-05-20,2020-05-20,원안가결,NaN,의안원문,제20대 (2016~2020) 제378회,NaN,행정안전위원회,NaN,NaN


In [202]:
# '제안일자'기준 정렬 후 합치기

BillDf_20 = BillDf_20.sort_values(by="제안일자")
BillDf_21 = BillDf_21.sort_values(by="제안일자")

BillDf = pd.concat([BillDf_20,BillDf_21]).drop(['Unnamed: 0','index'],axis=1)
BillDf.head()

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
25221,2000001,처,통일경제파주특별자치시의 설치 및 파주평화경제특별구역의 조성·운영과 지원에 관한 ...,의원,2016-05-30,2020-05-29,임기만료폐기,NaN,의안원문\r\n비용추계서 미첨부 사유서,제20대 (2016~2020) 제342회,['제안이유소득 양극화와 계층분화 심화ㆍ가계부채 심화ㆍ국가부채 급증ㆍ사회갈등 심화ㆍ...,외교통일위원회,"['박정(더불어민주당/朴釘)', '김경협(더불어민주당/金炅俠)', '노웅래(더불어민..."
25192,2000030,처,산업재해보상보험법 일부개정법률안(이완영의원 등 122인),의원,2016-05-30,2017-09-28,대안반영폐기,NaN,의안원문\r\n비용추계서,제20대 (2016~2020) 제342회,['■ 제안이유현행법은 사업주가 제공한 교통수단이나 그에 준하는 교통수단을 이용하는...,환경노동위원회,"['이완영(새누리당/李完永)', '강석진(새누리당/姜錫振)', '강석호(새누리당/姜..."
25191,2000031,처,파견근로자보호 등에 관한 법률 일부개정법률안(이완영의원 등 122인),의원,2016-05-30,2020-05-29,임기만료폐기,NaN,의안원문,제20대 (2016~2020) 제342회,['■ 제안이유동 법률이 제정된 이후 파견허용 업무가 한정적으로 운영되고 있어 급변...,환경노동위원회,"['이완영(새누리당/李完永)', '강석진(새누리당/姜錫振)', '강석호(새누리당/姜..."
25190,2000032,처,국가 사이버안보에 관한 법률안(이철우의원 등 122인),의원,2016-05-30,2020-05-29,임기만료폐기,NaN,의안원문,제20대 (2016~2020) 제342회,['제안이유2009년 7.7디도스 사건 이후 지속적인 사이버공격으로 청와대는 물론 ...,정보위원회,"['이철우(새누리당/李喆雨)', '강석진(새누리당/姜錫振)', '강석호(새누리당/姜..."
25189,2000033,처,행정규제기본법 폐지법률안(김광림의원 등 14인),의원,2016-05-30,2020-05-29,임기만료폐기,NaN,의안원문,제20대 (2016~2020) 제342회,['■ 제안이유 및 주요내용저출산？고령화 시대 및 저성장에 대응하고 국가경쟁력 강화...,정무위원회,"['김광림(새누리당/金光琳)', '강석진(새누리당/姜錫振)', '권석창(새누리당/權..."


In [203]:
BillDf.isna().sum()

의안번호          0
처리여부          0
의안명           0
제안자구분         0
제안일자          0
의결일자       4351
의결결과       4351
심사진행상태    25222
문서          246
제안회기          0
제안이유        557
소관위원회       119
제안자        1631
dtype: int64

## 전처리

#### 함수

In [204]:
# 전처리 함수 생성

def preprocessing(BillDf):
    
    
    # 결측값 처리
    
    BillDf = BillDf.fillna("")


    
    # [의안번호] ZZ로 시작하는 행 삭제 
    
    startZ = BillDf[BillDf['의안번호'].str.startswith('ZZ')]

    BillDf.drop(startZ.index, inplace=True)
    BillDf = BillDf.reset_index(drop=True)
    
    
    
    
    # [제안이유] 특수문자 및 '제안이유 및 주요내용', '제안이유', '주요내용' 문자열 삭제
    
    def clean_reason(df_col): 
        text = df_col[2:-2].replace('제안이유 및 주요내용','').replace('대안의 제안이유 및 주요내용','').replace('대안의 주요내용','').replace('대안의 제안이유','').replace('제안이유','').replace('주요내용','')
        return re.sub('[-=.#/:$}·,■?]', ' ', text)
    
    BillDf['제안이유'] = BillDf['제안이유'].apply(lambda x: clean_reason(x))
    
    

    
    
    # [의안명]뒤 대표 제안자 포함 괄호 삭제
    
    for i in range(len(BillDf)):
        BillDf['의안명'][i] = BillDf['의안명'][i].split('(')[0]
        
        
        
        
    # [의안명]에 '법률안' 포함하는 경우만 남기고 삭제 (법률안 -> 4254개 중 4007개) 
    
    BillDf.loc[BillDf['의안명'].str.endswith('법률안') == True, '법률안'] = 1
    BillDf.loc[BillDf['의안명'].str.endswith('법률안') == False, '법률안'] = 0
    BillDf = BillDf[BillDf['법률안'] == 1]
        
        
        
        
    # [제안자구분] 의원, 의장, 위원장 데이터만 남기고 삭제
    
    proposer_type_list = ['의원', '위원장', '의장']
    
    
    BillDf_new = BillDf[BillDf['제안자구분'].isin(proposer_type_list)]
    BillDf = BillDf_new.reset_index(drop = True)
    
    
    
    
    # [제안자] 스플릿, 제안 정당만 따와서 리스트로 변환, 중복제거 -> [제안정당] 칼럼 생성
    
    BillDf['제안자'] = BillDf['제안자'].str.split(',')
    BillDf['제안자'] = BillDf['제안자'].fillna("a") #object of type 'float' has no len() 오류 해결

    list1 = []

    for i in range(len(BillDf)):

        list2 = []

        if BillDf['제안자'][i] != "a":

            for j in range(len(BillDf['제안자'][i])):

                list2.append(BillDf['제안자'][i][j].split('(')[-1].split('/')[0])

            list1.append(list(set(list2)))

        else:
            list1.append(np.NaN)

    BillDf['제안정당'] = list1  
    
    
    
    
    # [제안자구분] '위원장'인 경우 제안정당 -> 더불어민주당으로 채우기
    
    for i in range(len(BillDf)):
        if BillDf['제안자구분'][i]=='위원장':
            BillDf['제안정당'][i]=[]
            BillDf['제안정당'][i].append('더불어민주당')

            

            
    # [소관위원회] 없는 행 제거
    
    noncomm = BillDf[BillDf['소관위원회'] == ''].index
    BillDf.drop(noncomm,inplace=True)
    BillDf = BillDf.reset_index(drop=True)
    
    noncomm = BillDf[BillDf['소관위원회'] == '[]'].index
    BillDf.drop(noncomm,inplace=True)
    BillDf = BillDf.reset_index(drop=True)
    
    
    
    
    # [의결결과] 공포, 미공포 분류
    
    proclaim_list = ['원안가결', '수정가결']
    reject_list = ['임기만료폐기', '대안반영폐기', '철회', '폐기', '부결', '수정안반영폐기']

    BillDf.loc[BillDf['의결결과'].isin(proclaim_list), '법률공포여부'] = '공포'
    BillDf.loc[BillDf['의결결과'].isin(reject_list), '법률공포여부'] = '미공포'
    
    
    
    # ['의안명']+[제안이유] -> ['텍스트'] 만들기
    
    
    BillDf['텍스트'] = ''
    
    for i in range(len(BillDf)):
        
        text = []
        
        text.append(BillDf['의안명'][i])
        text.append(BillDf['제안이유'][i])
    
        BillDf['텍스트'][i] = text
    
    
    
    # 결측값 처리
    
    BillDf = BillDf.fillna("")
    
    
    
    
    return BillDf

In [205]:
BillDf = preprocessing(BillDf)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [206]:
BillDf.head()

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,법률안,제안정당,법률공포여부,텍스트
0,2000030,처,산업재해보상보험법 일부개정법률안,의원,2016-05-30,2017-09-28 00:00:00,대안반영폐기,,의안원문\r\n비용추계서,제20대 (2016~2020) 제342회,현행법은 사업주가 제공한 교통수단이나 그에 준하는 교통수단을 이용하는 등 사업주...,환경노동위원회,"[['이완영(새누리당/李完永)', '강석진(새누리당/姜錫振)', '강석호(새누리...",1.0,[새누리당],미공포,"[ 산업재해보상보험법 일부개정법률안, 현행법은 사업주가 제공한 교통수단이나 그..."
1,2000031,처,파견근로자보호 등에 관한 법률 일부개정법률안,의원,2016-05-30,2020-05-29 00:00:00,임기만료폐기,,의안원문,제20대 (2016~2020) 제342회,동 법률이 제정된 이후 파견허용 업무가 한정적으로 운영되고 있어 급변하는 노동시...,환경노동위원회,"[['이완영(새누리당/李完永)', '강석진(새누리당/姜錫振)', '강석호(새누리...",1.0,[새누리당],미공포,"[ 파견근로자보호 등에 관한 법률 일부개정법률안, 동 법률이 제정된 이후 파견..."
2,2000032,처,국가 사이버안보에 관한 법률안,의원,2016-05-30,2020-05-29 00:00:00,임기만료폐기,,의안원문,제20대 (2016~2020) 제342회,2009년 7 7디도스 사건 이후 지속적인 사이버공격으로 청와대는 물론 언론？금융사...,정보위원회,"[['이철우(새누리당/李喆雨)', '강석진(새누리당/姜錫振)', '강석호(새누리...",1.0,[새누리당],미공포,"[ 국가 사이버안보에 관한 법률안, 2009년 7 7디도스 사건 이후 지속적인 사..."
3,2000033,처,행정규제기본법 폐지법률안,의원,2016-05-30,2020-05-29 00:00:00,임기만료폐기,,의안원문,제20대 (2016~2020) 제342회,저출산？고령화 시대 및 저성장에 대응하고 국가경쟁력 강화를 위하여 이 법에서 정...,정무위원회,"[['김광림(새누리당/金光琳)', '강석진(새누리당/姜錫振)', '권석창(새누리...",1.0,[새누리당],미공포,"[ 행정규제기본법 폐지법률안, 저출산？고령화 시대 및 저성장에 대응하고 국가경..."
4,2000034,처,청년고용촉진 특별법 일부개정법률안,의원,2016-05-30,2018-12-27 00:00:00,대안반영폐기,,의안원문\r\n비용추계서,제20대 (2016~2020) 제342회,2015년 12월말 15～29세 청년 실업률은 9 2%로 전체 실업률 3 6%의 3...,환경노동위원회,"[['박남춘(더불어민주당/朴南春)', '김현미(더불어민주당/金賢美)', '문희상...",1.0,[더불어민주당],미공포,"[ 청년고용촉진 특별법 일부개정법률안, 2015년 12월말 15～29세 청년 실업..."


In [207]:
BillDf.isna().sum()

의안번호      0
처리여부      0
의안명       0
제안자구분     0
제안일자      0
의결일자      0
의결결과      0
심사진행상태    0
문서        0
제안회기      0
제안이유      0
소관위원회     0
제안자       0
법률안       0
제안정당      0
법률공포여부    0
텍스트       0
dtype: int64

In [208]:
BillDf_ = BillDf.copy()

#### '코로나'키워드를 포함하는 의안 가져오기

In [209]:
contain_covid = []

for i in tqdm(range(len(BillDf_)), desc = "코로나"):
    if "코로나" in str(BillDf_['텍스트'][i]):
        contain_covid.append(True)
    else:
        contain_covid.append(False)
        
Covid = BillDf_[contain_covid]
Covid['텍스트'][:2]


코로나: 100%|██████████| 26069/26069 [00:00<00:00, 35331.65it/s]


22072    [  감염병의 예방 및 관리에 관한 법률 일부개정법률안, 최근 신종 코로나바이러스감...
22074    [  검역법 일부개정법률안, 현행법은 보건복지부장관이 출입국의 금지 또는 정지 요청...
Name: 텍스트, dtype: object

In [210]:
Covid = Covid.reset_index(drop=True)

In [211]:
len(Covid)

415

#### 한글 제외 문자 제거

In [212]:
def onlyKorean(textlist):
    
    for i in tqdm(range(len(textlist)), desc = "텍스트전처리"):
        
        textlist[i] = re.sub('[^가-힣\s]','',str(textlist[i]))
    
    return textlist

Covid['텍스트'] = onlyKorean(Covid['텍스트'])

텍스트전처리:   0%|          | 0/415 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
텍스트전처리: 100%|██████████| 415/415 [00:00<00:00, 6993.17it/s]


## 2. 형태소 분석기 결정

In [213]:
def hannanumtest(textlist):

        hannanum = Hannanum()
        nounlist = []
        
        for i in range(len(textlist)):
            n = re.sub('[-=.#/:$}·,■?]','', str(textlist[i]))
            k = re.sub('[0-9]','',n)
            nounlist.append(hannanum.nouns(k))


        return nounlist


def kkmatest(textlist):

        kkma = Kkma()
        nounlist = []
        
        for i in range(len(textlist)):
            n = re.sub('[-=.#/:$}·,■?]','',str(textlist[i]))
            k = re.sub('[0-9]','',n)
            nounlist.append(kkma.nouns(k))


        return nounlist


    
def okttest(textlist):

        okt = Okt()
        nounlist = []
        
        for i in range(len(textlist)):
            n = re.sub('[-=.#/:$}·,■?]','',str(textlist[i]))
            k = re.sub('[0-9]','',n)
            nounlist.append(okt.nouns(k))


        return nounlist

    
    
def komorantest(textlist):

        komoran = Komoran()
        nounlist = []
        
        for i in range(len(textlist)):
            n = re.sub('[-=.#/:$}·,■?]','',str(textlist[i]))
            k = re.sub('[0-9]','',n)
            nounlist.append(komoran.nouns(k))


        return nounlist

In [214]:
hannanum_result = hannanumtest(Covid['텍스트'][:3])
kkma_result = kkmatest(Covid['텍스트'][:3])
okt_result = okttest(Covid['텍스트'][:3])
komoran_result = komorantest(Covid['텍스트'][:3])

print('hannanum\n',hannanum_result[:3])
print('\nkkma\n',kkma_result[:3])
print('\nokt\n',okt_result[:3])
print('\nkomoran\n',komoran_result[:3])

hannanum
 [['감염병', '예방', '관리', '법률', '일부개정법률안', '신종', '코로나바이러스감염증', '의', '국내', '유입', '확진환자', '명', '조사대상', '유증상자', '수십명', '달', '상황', '일부', '확진환자', '며칠간', '도심일대', '활보', '것', '확인', '감염자', '발생', '우려', '이', '보건복지부', '신종', '코로나바이러스감염증', '확', '대비', '감염병', '위기경보', '단계', '주의', '경계', '격상', '신종', '코로나바이러스감염증', '중앙사고수습본부', '설치', '가동', '신종', '코로나바이러스감염증', '경우', '현행', '법령', '이', '명시적', '규정하', '감염병', '관리', '조치', '차질', '빚', '수', '점', '이', '규정', '필요', '이', '현행법', '규정', '급감염병', '신종', '코로나바이러스감염증', '을', '추가', '제조제호처목', '신설'], ['검역법', '일부개정법률안', '현행법', '보건복지부장관', '출입국', '금지', '정지', '요청', '규정상', '검역감염병', '환자등', '검역감염병', '의심자', '법무부장관', '출국', '입국', '금지', '정지', '요청', '수', '이', '환자', '감염의심자', '단위', '출국', '입국', '금지', '정지', '요청', '것', '뿐', '검역감염병', '발생지역', '입국', '지역', '경유', '입국', '자', '경우', '잠복기', '증상', '입국', '시', '의심', '실제', '감염자', '수', '이', '보건복지부장관', '공중위생상', '위해', '염려', '인정', '경우', '세계보건기구', '공중보건', '위기관리', '대상', '감염병', '환자', '발생', '지역', '입국', '이', '경유', '입국', '외국인', '출국', '입국', '정지', '요청', '수', '근거조항', '마련

**=> Hannanum을 사용하자!**

## 3. 토큰화

#### 함수

In [215]:
# 토큰생성

def get_tokens(text):
    
    hannanum = Hannanum()
        
    return [i for i in hannanum.nouns(text) if len(i) > 1] if text else []
    

#### 토큰화 및 토큰 개수 count

In [218]:
Covid['토큰'] = Covid['텍스트'].map(get_tokens)

In [219]:
Covid['명사수'] = Covid['토큰'].map(len)

In [220]:
Covid.head()

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자,법률안,제안정당,법률공포여부,텍스트,토큰,명사수
0,2024549,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,의원,2020-01-28,2020-02-26 00:00:00,대안반영폐기,,의안원문,제20대 (2016~2020) 제375회,최근 신종 코로나바이러스감염증(2019 novel Coronavirus 2019 ...,보건복지위원회,"[['유의동(새로운보수당/兪義東)', '권성동(자유한국당/權性東)', '김상훈(...",1.0,"[미래를향한전진4.0, 자유한국당, 바른미래당, 새로운보수당]",미공포,감염병의 예방 및 관리에 관한 법률 일부개정법률안 최근 신종 코로나바이러스감염증...,"[감염병, 예방, 관리, 법률, 일부개정법률안, 신종, 코로나바이러스감염증, 국내,...",59
1,2024556,처,검역법 일부개정법률안,의원,2020-01-29,2020-05-29 00:00:00,임기만료폐기,,의안원문,제20대 (2016~2020) 제375회,현행법은 보건복지부장관이 출입국의 금지 또는 정지 요청 규정상 검역감염병 환자등 또...,보건복지위원회,"[['원유철(자유한국당/元裕哲)', '강석호(자유한국당/姜碩鎬)', '김성태(자...",1.0,"[새로운보수당, 무소속, 자유한국당]",미공포,검역법 일부개정법률안 현행법은 보건복지부장관이 출입국의 금지 또는 정지 요청 규...,"[검역법, 일부개정법률안, 현행법, 보건복지부장관, 출입국, 금지, 정지, 요청, ...",74
2,2024555,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,의원,2020-01-29,2020-02-26 00:00:00,대안반영폐기,,의안원문,제20대 (2016~2020) 제375회,현행법은 감염병환자등이 발생하였거나 발생할 우려가 있는 경우 육군ㆍ해군ㆍ공군 소속 ...,보건복지위원회,"[['원유철(자유한국당/元裕哲)', '강석호(자유한국당/姜碩鎬)', '김성태(자...",1.0,"[새로운보수당, 무소속, 자유한국당]",미공포,감염병의 예방 및 관리에 관한 법률 일부개정법률안 현행법은 감염병환자등이 발생하...,"[감염병, 예방, 관리, 법률, 일부개정법률안, 현행법, 감염병환자, 발생, 발생,...",59
3,2024582,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,의원,2020-02-06,2020-02-26 00:00:00,대안반영폐기,,의안원문\n비용추계서 미첨부 사유서,제20대 (2016~2020) 제375회,세계적으로 신종감염병의 수가 지난 60년간 4배 이상 늘어나면서 감염병 유행이 전 ...,보건복지위원회,"[['기동민(더불어민주당/奇東旻)', '강훈식(더불어민주당/姜勳植)', '금태섭...",1.0,[더불어민주당],미공포,감염병의 예방 및 관리에 관한 법률 일부개정법률안 세계적으로 신종감염병의 수가 ...,"[감염병, 예방, 관리, 법률, 일부개정법률안, 세계적, 신종감염병, 이상, 감염병...",142
4,2024584,처,감염병의 예방 및 관리에 관한 법률 일부개정법률안,의원,2020-02-06,2020-02-26 00:00:00,대안반영폐기,,의안원문,제20대 (2016~2020) 제375회,최근 신종 코로나바이러스 감염병과 관련하여 의료기관은 중국 입국자 및 접촉자 관련 ...,보건복지위원회,"[['허윤정(더불어민주당/許允貞)', '기동민(더불어민주당/奇東旻)', '김상희...",1.0,"[더불어민주당, 무소속]",미공포,감염병의 예방 및 관리에 관한 법률 일부개정법률안 최근 신종 코로나바이러스 감염...,"[감염병, 예방, 관리, 법률, 일부개정법률안, 신종, 코로나바이러스, 감염병, 관...",91


In [221]:
Covid['명사수'].describe()

count    415.000000
mean      90.978313
std       64.557333
min       29.000000
25%       61.000000
50%       75.000000
75%       96.500000
max      784.000000
Name: 명사수, dtype: float64

In [222]:
len(Covid)

415

In [223]:
# csv로 저장
Covid.to_csv('[코로나] tocken.csv', encoding='utf-8-sig')